---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [18]:
#%matplotlib notebook
#import matplotlib.pyplot as plt
#import time

def graph_identification():
    
    # Your Code Here
    
    '''
    # initialize subplots
    fig, axarr = plt.subplots(5, figsize=(5, 5))
    plt.tight_layout()
    
    # iterate over the list of graphs
    for i in range(len(P1_Graphs)):
        
        # variable for the graph
        G = P1_Graphs[i]
        print(G.name)

        # list all the degrees
        degrees = G.degree()

        # list all the degree values
        degree_values = sorted(set(degrees.values()))

        # create histogram
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
        
        # plot name
        plot_name = "Graph: " + str(i + 1) + " Degree vs fration of nodes "
        
        # set the title of each subplot
        axarr[i].set_title(plot_name)
        
        # et common labels
        axarr[i].set_xlabel('Degree')
        axarr[i].set_ylabel('Fraction of Nodes')

        # plot the histogram
        pd.DataFrame(histogram).plot.bar(ax=axarr[i])
        
    plt.show()  
    '''
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H' ] # Your Answer Here

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [3]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [6]:
from sklearn.metrics import average_precision_score, precision_recall_curve, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def salary_predictions():
    
    # Your Code Here
    #---------------------------------------------STRUCTURING THE DATAFRAME FOR ANALYSIS----------------------------------#
    # Initialize the dataframe, using the nodes as the index
    df = pd.DataFrame(index=G.nodes())
    
    # create additional features
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['clustering_coeff'] = pd.Series(nx.clustering(G))
    df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G))
    df['betweenness_centrality'] = pd.Series(nx.betweenness_centrality(G))

    # add attributes into the dataframe
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    # train set
    train_df = df.dropna(subset = ['ManagementSalary'])
    
    # extract all the employees without dat on ManagementSalary
    test_df = df[df['ManagementSalary'].isnull()]
    
    # ---------------------------------------CREATE A MACHINE LEARNING MODEL TO PREDICT SALARY--------------------------#
    
    # split the data that has salary data from the rest
    X_train_df = train_df.iloc[:,:-1]
    y_train_df = train_df.iloc[:,-1]
    X_test_df = test_df.iloc[:,:-1]
    
    # create a train and test split
    X_train, X_test, y_train, y_test = train_test_split(X_train_df, y_train_df, random_state=0)
    
    # initiallise the scaler
    scaler = StandardScaler().fit(X_train)
    
    # scale the datasets
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    X_test_df_2 = scaler.transform(X_test_df)

    # initialise the classifier
    clf = LogisticRegression()

    # Train the model using the training sets
    clf.fit(X_train, y_train)

    # predict the y labels
    y_pred = pd.DataFrame(clf.predict_proba(X_test))

    # calculate the AUC
    fpr, tpr, threshold = roc_curve(y_test, y_pred.iloc[:,-1])
    roc_auc = auc(fpr, tpr)
 
    # predict the y labels of the test.csv set
    y_salaries = pd.DataFrame(clf.predict_proba(X_test_df_2))
 
    # store the final result
    result = pd.Series(data = np.array(y_salaries.iloc[:,-1]), index = X_test_df.index, name=None)
    
    return result   # Your Answer Here

salary_predictions()

1       0.057182
2       0.719403
5       0.999905
8       0.119456
14      0.181714
18      0.234906
27      0.257620
30      0.311309
31      0.106237
34      0.106899
37      0.041988
40      0.090865
45      0.144858
54      0.232902
55      0.434130
60      0.064795
62      0.999557
65      0.860020
77      0.057775
79      0.104019
97      0.074049
101     0.029122
103     0.182264
108     0.093549
113     0.434030
122     0.010525
141     0.429043
142     0.871463
144     0.095903
145     0.618972
          ...   
913     0.016115
914     0.045881
915     0.000043
918     0.118052
923     0.006891
926     0.076843
931     0.031534
934     0.003794
939     0.001342
944     0.000075
945     0.012835
947     0.173209
950     0.102938
951     0.010459
953     0.007709
959     0.000119
962     0.000057
963     0.129842
968     0.085865
969     0.091862
974     0.094388
984     0.003079
987     0.122824
989     0.087235
991     0.134531
992     0.000232
994     0.000178
996     0.0000

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [4]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [15]:
from sklearn.metrics import average_precision_score, precision_recall_curve, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def new_connections_predictions():
    
    # copy the data frame
    df = future_connections.iloc[:,:-1]
    
    # add measures  as columns
    df['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['jaccard_coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    # df['resource_allocation_index'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
    df['adamic_adar_index'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
    
    # add in the label column
    df['Future Connection'] = future_connections['Future Connection']
    
    # ---------------------------------------CREATE A MACHINE LEARNING MODEL TO PREDICT CONNECTIONS-------------------------#
    # train set
    train_df = df.dropna(subset = ['Future Connection'])
    
    # extract all the employees without dat on ManagementSalary
    test_df = df[df['Future Connection'].isnull()]
    
    # split the data that has salary data from the rest
    X_train_df = train_df.iloc[:,:-1]
    y_train_df = train_df.iloc[:,-1]
    X_test_df = test_df.iloc[:,:-1]
    
    # create a train and test split
    X_train, X_test, y_train, y_test = train_test_split(X_train_df, y_train_df, random_state=0)
    
    # initiallise the scaler
    scaler = StandardScaler().fit(X_train)
    
    # scale the datasets
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    X_test_df_2 = scaler.transform(X_test_df)

    # initialise the classifier
    clf = LogisticRegression()

    # Train the model using the training sets
    clf.fit(X_train, y_train)

    # predict the y labels
    y_pred = pd.DataFrame(clf.predict_proba(X_test))

    # calculate the AUC
    fpr, tpr, threshold = roc_curve(y_test, y_pred.iloc[:,-1])
    roc_auc = auc(fpr, tpr)
 
    # predict the y labels of the test.csv set
    y_connection_pred = pd.DataFrame(clf.predict_proba(X_test_df_2))
 
    # store the final result
    result = pd.Series(data = np.array(y_connection_pred.iloc[:,-1]), index = X_test_df.index, name=None)
    
    return result   # Your Answer Here
                                      
new_connections_predictions() 


0.90803290963237671